<a href="https://colab.research.google.com/github/GGpark1/Deep_Learning_Study/blob/master/CNN_practice_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#모듈 import

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

In [2]:
#시드 고정

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
#데이터셋 저장

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [31]:
X_train.shape

(40000, 32, 32, 3)

In [4]:
#이미지 정규화

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [5]:
#검증 데이터 생성

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

In [6]:
#신경망 모델 구축

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu')) #필터의 갯수는 32개, 필터의 크기는 3*3, Convolution 진행, 출력함수는 ReLu, 입력값과 출력값의 크기가 같도록 padding
model.add(MaxPooling2D(2,2)) #2*2 단위로 Max Pooling 진행
model.add(Conv2D(64, (3,3))) #필터의 갯수는 54개, 크기는 3*3
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu')) #MaxPooling은 생략. 이미지 전체의 특징을 추출하지 않도록 Pooling을 멈춤
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [7]:
#모델 compile

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
#모델 훈련

model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)

Epoch 1/10
313/313 [==============================] - 12s 6ms/step - loss: 1.5342 - accuracy: 0.4467 - val_loss: 1.2551 - val_accuracy: 0.5605
Epoch 2/10
313/313 [==============================] - 2s 5ms/step - loss: 1.1406 - accuracy: 0.5964 - val_loss: 1.0402 - val_accuracy: 0.6375
Epoch 3/10
313/313 [==============================] - 2s 5ms/step - loss: 0.9580 - accuracy: 0.6660 - val_loss: 0.9726 - val_accuracy: 0.6572
Epoch 4/10
313/313 [==============================] - 2s 5ms/step - loss: 0.8404 - accuracy: 0.7077 - val_loss: 0.9048 - val_accuracy: 0.6826
Epoch 5/10
313/313 [==============================] - 2s 5ms/step - loss: 0.7457 - accuracy: 0.7404 - val_loss: 0.8837 - val_accuracy: 0.6922
Epoch 6/10
313/313 [==============================] - 2s 5ms/step - loss: 0.6636 - accuracy: 0.7666 - val_loss: 0.8584 - val_accuracy: 0.7075
Epoch 7/10
313/313 [==============================] - 2s 5ms/step - loss: 0.5839 - accuracy: 0.7981 - val_loss: 0.8505 - val_accuracy: 0.7101
Epoch

In [9]:
#모델 평가

model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.9661 - accuracy: 0.7130 - 611ms/epoch - 2ms/step


[0.9661082029342651, 0.7129999995231628]

## 전이학습(VGG)

In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [11]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2)

In [14]:
#전이학습 모델 load
#출력층은 포함하지 않음
#합성곱층만 로드

pretrained_model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [43]:
model = Sequential()
model.add(pretrained_model)
model.add(GlobalAveragePooling2D()) #flatten과 비슷한 효과 but 채널의 갯수는 그대로 #(16, 7, 7, 64) into (16, 64)
#model.add(Flatten()) #flatten은 채널도 펴버림 #(16, 28, 28, 3) into (16, 2352) #input에서 채널 수가 포함되어 있는 경우 flatten은 작동하지 않음 -> Dense층에서 채널 차원을 요구함
model.add(Dense(128,activation='relu')) 
model.add(Dense(10,activation='softmax'))

In [44]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d_4   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_19 (Dense)            (None, 128)               65664     
                                                                 
 dense_20 (Dense)            (None, 10)                1290      
                                                                 
Total params: 14,781,642
Trainable params: 14,781,642
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)

Epoch 1/10
313/313 [==============================] - 12s 37ms/step - loss: 0.2390 - accuracy: 0.9290 - val_loss: 0.8314 - val_accuracy: 0.7920
Epoch 2/10
313/313 [==============================] - 11s 35ms/step - loss: 0.1421 - accuracy: 0.9572 - val_loss: 0.8379 - val_accuracy: 0.7989
Epoch 3/10
313/313 [==============================] - 11s 35ms/step - loss: 0.1242 - accuracy: 0.9610 - val_loss: 0.8940 - val_accuracy: 0.7946
Epoch 4/10
313/313 [==============================] - 11s 36ms/step - loss: 0.1299 - accuracy: 0.9607 - val_loss: 0.9374 - val_accuracy: 0.7872
Epoch 5/10
313/313 [==============================] - 11s 35ms/step - loss: 0.1230 - accuracy: 0.9628 - val_loss: 0.8940 - val_accuracy: 0.8003
Epoch 6/10
313/313 [==============================] - 11s 35ms/step - loss: 0.1031 - accuracy: 0.9688 - val_loss: 0.9496 - val_accuracy: 0.7941
Epoch 7/10
313/313 [==============================] - 11s 35ms/step - loss: 0.1039 - accuracy: 0.9671 - val_loss: 0.9974 - val_accuracy: